In [16]:
import pandas as pd
import nltk
import spacy

In [13]:
df = pd.read_csv('/Users/pujaarajan/Documents/GitHub/nlp_fib/data/raw/enron_05_17_2015_with_labels_v2_100K_chunk_1_of_6.csv', usecols= [*range(2, 6),13], dtype={13:str})

In [14]:
df.head()

,Date,From,To,Subject,content
0,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Here is our forecast
1,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Traveling to have a business meeting takes the...
2,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,test successful. way to go!!!
3,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,"Randy, Can you send me a schedule of the salar..."
4,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Let's shoot for Tuesday at 11:45.


In [89]:
nlp = spacy.load('en')

#punctuations = string.punctuation

def preprocess_text(doc):
    """ Preprocess text. Tokenize, lowercase, and remove punctuation and stopwords """
    stopwords = spacy.lang.en.STOP_WORDS
    # Tokenize, remove punctuation, symbols (#), stopwords
    doc = [tok.text for tok in doc if (tok.text not in stopwords and tok.pos_ != "PUNCT" and tok.pos_ != "SYM" and tok.pos_ != "X")]
    # Lowercase tokens
    doc = [tok.lower() for tok in doc]
    doc = ' '.join(doc)
    return nlp.make_doc(doc)

nlp.add_pipe(preprocess_text, name='preprocesser', after='tagger')
print(nlp.pipe_names) # ['tagger', 'preprocesser','parser', 'ner']

['tagger', 'preprocesser', 'parser', 'ner']


# Tokenize

In [87]:
spacy_tokenized_emails = []
for email in sample_emails:
    sentence_tokens = []
    doc = nlp(email[0])
    for token in doc:
        sentence_tokens.append(token.text)
    spacy_tokenized_emails.append(sentence_tokens)

In [91]:
import torch, torch.nn as nn
from torch.autograd import Variable

text = ['BOS', 'How', 'are', 'you', 'EOS']
seq_len = len(text)
batch_size = 1
embedding_size = 1
hidden_size = 1
output_size = 1

random_input = Variable(
    torch.FloatTensor(seq_len, batch_size, embedding_size).normal_(), requires_grad=False)

bi_rnn = torch.nn.RNN(
    input_size=embedding_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

bi_output, bi_hidden = bi_rnn(random_input)

# stagger
forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
staggered_output = torch.cat((forward_output, backward_output), dim=-1)

linear = nn.Linear(hidden_size * 2, output_size)

# only predict on words
labels = random_input[1:-1]

# for language models, use cross-entropy :)
loss = nn.MSELoss()
output = loss(linear(staggered_output), labels)

In [94]:
labels

tensor([[[-0.9892]],

        [[ 0.6772]],

        [[-2.7760]]])

# Old Code

# NLTK Tokenizing

In [90]:
from nltk.tokenize import word_tokenize
tokenized_emails = []
for email in sample_emails:
    tokens = word_tokenize(email[0])
    tokenized_emails.append(tokens)
print(tokenized_emails[0:5])

[['Here', 'is', 'our', 'forecast'], ['Traveling', 'to', 'have', 'a', 'business', 'meeting', 'takes', 'the', 'fun', 'out', 'of', 'the', 'trip', '.', 'Especially', 'if', 'you', 'have', 'to', 'prepare', 'a', 'presentation', '.', 'I', 'would', 'suggest', 'holding', 'the', 'business', 'plan', 'meetings', 'here', 'then', 'take', 'a', 'trip', 'without', 'any', 'formal', 'business', 'meetings', '.', 'I', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'a', 'trip', 'is', 'even', 'desired', 'or', 'necessary', '.', 'As', 'far', 'as', 'the', 'business', 'meetings', ',', 'I', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'different', 'groups', 'about', 'what', 'is', 'working', 'and', 'what', 'is', 'not', '.', 'Too', 'often', 'the', 'presenter', 'speaks', 'and', 'the', 'others', 'are', 'quiet', 'just', 'waiting', 'for', 'their', 'turn', '.', 'The', 'meetings', 'might', 'be', 'better', 'if', 'h